In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc

from IPython.display import display

/public/workspace/licanchengup/apps/miniconda3/envs/csMAHN/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
p_link = Path('/public/workspace/licanchengup/link')
p_test = p_link.joinpath('test')
p_root = Path('/public/workspace/licanchengup/link/db/came/dataset/Retina')
None if str(p_test) in sys.path else sys.path.append(str(p_test))

In [3]:
from func import h5ad_to_mtx, group_agg


[func]----------------------------------
group_agg
rm_rf
get_path_varmap
get_type_counts_info		aligned
aligned_type			aligned
test_came_and_csMAHN
h5ad_to_mtx
load_adata

[result tools]--------------------------
get_test_result_df
show_umap
get_res_obs
get_publi_head

[result tools][re plot umap]-----------
get_df_color			Convenient
> [dssp umap]
export_legend
dssp_plot_umap_legend_sp	Convenient
dssp_plot_umap_legend_cell_type	Convenient
dssp_plot_umap
dssp_get_umap_name
dssp_plot_save_umap		Convenient
> [umap]
get_default_df_color
plot_umap
get_umap_name
plot_save_umap			Convenient
----------------------------------------
    


In [4]:
df_path = pd.DataFrame({
    'path': Path('.').iterdir()
})
df_path = df_path[df_path['path'].apply(lambda x: x.match('*GSE*'))]
df_path['name'] = df_path['path'].apply(lambda x: x.stem)
df_path['gse_id'] = df_path['name'].str.extract(
    '^(GSE\\d+)_', expand=False)
df_path['type'] = ''
df_path['type'] = df_path['type'].mask(
    df_path['name'].str.contains('gene'), 'var')
df_path['type'] = df_path['type'].mask(
    df_path['name'].str.contains('annotations'), 'obs')
df_path['type'] = df_path['type'].mask(
    df_path['name'].str.contains('counts'), 'mtx')
df_path = df_path.pivot(index='gse_id', columns='type', values='path')
df_path

type,mtx,obs,var
gse_id,,,
GSE137537,GSE137537_counts.mtx.gz,GSE137537_sample_annotations.tsv.gz,GSE137537_gene_names.txt.gz
GSE137846,GSE137846_Seq-Well_counts.mtx.gz,GSE137846_Seq-Well_sample_annotations.txt.gz,GSE137846_Seq-Well_gene_names.txt.gz


In [5]:
res = {}
for _i, _row in df_path.iterrows():
    pass
_row

type
mtx                GSE137846_Seq-Well_counts.mtx.gz
obs    GSE137846_Seq-Well_sample_annotations.txt.gz
var            GSE137846_Seq-Well_gene_names.txt.gz
Name: GSE137846, dtype: object

In [6]:
_row = df_path.loc['GSE137537', :]
adata = sc.read_mtx(_row['mtx']).T
df_obs = pd.read_csv(_row['obs'], sep='\t', index_col=0)
# print(*df_obs.columns,sep='\n')
# df_obs['tissue'] = df_obs['tissue'].map({
#     'MR':'Macula-Retina',
#     'PR':'Peripheral retina'
# })
df_obs = df_obs.loc[:, ~df_obs.columns.str.match('^scater_')]
# display(adata,df_obs.head(2),df_obs.shape,df_var.head(2),df_var.shape)
display(
    group_agg(
        df_obs, [
            'individual', 'tissue'], {
                'tissue': ['count']}, reindex=False))
df_obs['_batch'] = df_obs.apply(
    lambda _row: "{individual};{tissue}".format(
        **_row), axis=1)
display(df_obs['_batch'].value_counts())
df_obs = df_obs.loc[:, '_batch,individual,tissue,Labels,x,y'.split(',')]

df_var = pd.read_csv(_row['var'], index_col=0, header=None, names=[''])
adata.var = df_var
adata.var_names_make_unique()
adata.obs = df_obs
adata.obs_names_make_unique()
display(
    adata,
    adata.obs.head(2),
    adata.obs.shape,
    adata.var.head(2),
    adata.var.shape)
res.update({
    _row.name: adata
})
del _row, adata, df_obs, df_var

tissue
                   count
individual tissue       
555        MR       1765
           PR       2505
574        MR2      5121
           PR2      4137
607        MR3      3119
           PR3      3444

_batch
574;MR2    5121
574;PR2    4137
607;PR3    3444
607;MR3    3119
555;PR     2505
555;MR     1765
Name: count, dtype: int64

AnnData object with n_obs × n_vars = 20091 × 19719
    obs: '_batch', 'individual', 'tissue', 'Labels', 'x', 'y'

,_batch,individual,tissue,Labels,x,y
Barcode,,,,,,
AAACCCAAGTTACTCG-1,574;MR2,574,MR2,Rods,1.196409,0.093193
AAACCCAGTGGAACAC-1,574;MR2,574,MR2,Rods,1.336815,0.444690


(20091, 6)

""
""
RP11-34P13.7
FO538757.2


(19719, 0)

In [7]:
_row = df_path.loc['GSE137846', :]

adata = sc.read_mtx(_row['mtx']).T
df_obs = pd.read_csv(_row['obs'], sep='\t')
# print(*df_obs.columns,sep='\n')
# df_obs['tissue'] = df_obs['tissue'].map({
#     'MR':'Macula-Retina',
#     'PR':'Peripheral retina'
# })
df_obs = df_obs.loc[:, ~df_obs.columns.str.match('^scater_')]
# display(adata,df_obs.head(2),df_obs.shape,df_var.head(2),df_var.shape)
display(
    group_agg(
        df_obs, [
            'individual', 'tissue'], {
                'tissue': ['count']}, reindex=False))
df_obs['_batch'] = df_obs.apply(
    lambda _row: "{individual};{tissue}".format(
        **_row), axis=1)
display(df_obs['_batch'].value_counts())
df_obs = df_obs.loc[:, '_batch,individual,tissue,Labels,x,y'.split(',')]

df_var = pd.read_csv(_row['var'], index_col=0, header=None, names=[''])

adata.var = df_var
adata.var_names_make_unique()
adata.obs = df_obs
adata.obs_names_make_unique()
display(
    adata,
    adata.obs.head(2),
    adata.obs.shape,
    adata.var.head(2),
    adata.var.shape)
res.update({
    _row.name: adata
})
# del _row,adata,df_obs,df_var

tissue
                   count
individual tissue       
time_28hrs MR        470
time_6hrs  MR        798
           PR        490
time_8hrs  MR        466
           PR       1024

_batch
time_8hrs;PR     1024
time_6hrs;MR      798
time_6hrs;PR      490
time_28hrs;MR     470
time_8hrs;MR      466
Name: count, dtype: int64

AnnData object with n_obs × n_vars = 3248 × 18219
    obs: '_batch', 'individual', 'tissue', 'Labels', 'x', 'y'

,_batch,individual,tissue,Labels,x,y
GCCGTGGTATCA_MR8hrs,time_8hrs;MR,time_8hrs,MR,BPs,0.613604,-0.167617
GCCAGTAAACCC_MR8hrs,time_8hrs;MR,time_8hrs,MR,HCs,-0.431396,-0.593605


(3248, 6)

""
""
A1BG-AS1
A2M


(18219, 0)

In [8]:
for k, v in res.items():
    print(k.ljust(50, '-'))
    display(v.obs.index.is_unique, v.var.index.is_unique)

GSE137537-----------------------------------------


True

True

GSE137846-----------------------------------------


True

True

In [9]:
adata = sc.concat(res, index_unique=';')
adata.obs['pre_ct'] = adata.obs['Labels']
adata.obs['cell_type'] = adata.obs['Labels'].map(lambda k: {
    'ACs': 'AC',
    'Endo': 'V/E cell',
    'Macroglia': 'Macroglial cell',
    'Microglia': 'Microglia cell',
    'HCs': 'HC',
    'Rods': 'Rods',
    'Cones': 'Cones',
    'RGCs': 'RGC',
    # ?
    'BPs': 'BPs',
}.setdefault(k, k))
display(adata.obs.head(2),adata.obs.shape)
h5ad_to_mtx(adata, p_root.joinpath('Menon_seqwell'))

,_batch,individual,tissue,Labels,x,y,pre_ct,cell_type
AAACCCAAGTTACTCG-1;GSE137537,574;MR2,574,MR2,Rods,1.196409,0.093193,Rods,Rods
AAACCCAGTGGAACAC-1;GSE137537,574;MR2,574,MR2,Rods,1.336815,0.444690,Rods,Rods


(23339, 8)

frist 10 data.X nonzero elements:
 [[2 1 1 1 1 1 1 1 1 1]]
[out] /public/workspace/licanchengup/link/db/came/dataset/Retina/Menon_seqwell




|||
|:-|:-|
|rod and cone photoreceptors|视杆和视锥光感受器|
|bipolar cells|双极细胞|
|amacrine cells|无长突细胞|
|horizontal cells|水平细胞|
|microglia|小胶质细胞|
|retinal ganglion cells|视网膜神经节细胞|
|vascular cells|血管细胞|
|Müller glia|米勒胶质细胞|
|astrocytes|星形胶质细胞|











In [10]:
display(adata.obs.index.is_unique,adata.var.index.is_unique)

True

True

In [11]:
for x in adata.obs['Labels'].unique():
    print("'{}':'',".format(x))

'Rods':'',
'ACs':'',
'Macroglia':'',
'BPs':'',
'HCs':'',
'Cones':'',
'RGCs':'',
'Endo':'',
'Microglia':'',
